In [1]:
"""
!pip install numpy==1.25.2
!pip install torch==2.7.0
!pip install torchvision==0.22.0
!pip install gym_super_mario_bros==7.4.0
!pip install gym==0.26.2
!pip install nes_py==8.2.1
!pip install gymnasium==0.29.1
!pip install opencv-python==4.11.0.86
"""

'\n!pip install numpy==1.25.2\n!pip install torch==2.7.0\n!pip install torchvision==0.22.0\n!pip install gym_super_mario_bros==7.4.0\n!pip install gym==0.26.2\n!pip install nes_py==8.2.1\n!pip install gymnasium==0.29.1\n!pip install opencv-python==4.11.0.86\n'

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import torch,gym,random
from dataclasses import dataclass
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation,FrameStack,ResizeObservation
from gym.vector import AsyncVectorEnv
import numpy as np

@dataclass(frozen=False)
class hypers:
    seed : int = 42
    lambda_ : float = 1.0
    gamma : float = 0.99
    epsilon : float = 0.2
    lr : float = 1e-4
    critic_coeff : float = 5e-1
    policy_ceoff : int = 3e3
    entropy_coeff : float = 1e-2
    skip_frame : int = 4
    num_stack : int  = 4
    obs_shape : tuple[int,int] = (100,100) # observation shape
    num_env : int = 5 # 30 
    num_game : int = 2_400
    batchsize : int = 100 # 512 
    minibatch : int = 16
    optim_steps : int = 10
    # 30*1400*512 ~ 20 M steps, training target
    device = torch.device(
        "cuda" if torch.cuda.is_available() else "cpu"
    )

configs = hypers()

torch.manual_seed(configs.seed)
random.seed(configs.seed)
np.random.seed(configs.seed)
torch.cuda.manual_seed(configs.seed)

In [3]:
class CustomEnv(gym.Wrapper): 
        def __init__(self,env,skip):
            super().__init__(env)
            self.skip = skip
            
        def step(self, action):
            total_reward = 0  
            for _ in range(self.skip):
                obs,reward,done,truncared,info = self.env.step(action)
                total_reward += reward
                if done:
                    self.reset()
                    return obs,(total_reward/10.),done,truncared,info
            return obs,(total_reward/10.),done,truncared,info

        def reset(self, **kwargs):
            obs,info = self.env.reset(**kwargs)
            return obs,info

def make_env():
    def env():
        x = gym_super_mario_bros.make("SuperMarioBros-v0", apply_api_compatibility=True)
        x = JoypadSpace(x, SIMPLE_MOVEMENT)
        x = ResizeObservation(x, configs.obs_shape)
        x = CustomEnv(x, configs.skip_frame)
        x = GrayScaleObservation(x, keep_dim=True)
        x = FrameStack(x, configs.num_stack) 
        return x
    return AsyncVectorEnv([env for _ in range(configs.num_env)])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.LazyConv2d(32,1,1,0)
        self.conv2 = nn.LazyConv2d(32,3,2,2)
        self.conv3 = nn.LazyConv2d(32,3,2,2)
        self.conv4 = nn.LazyConv2d(32,3,2,2)
        self.output = nn.LazyLinear(80)

        self.policy_head = nn.LazyLinear(7)
        self.value_head = nn.LazyLinear(1)
        self.optim = torch.optim.Adam(self.parameters(),lr=configs.lr)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = torch.flatten(x,start_dim=1) # -> torch.Size([32, 169])
        x = F.relu(self.output(x))
        policy_output = self.policy_head(x)
        value_output = self.value_head(x)
        return F.softmax(policy_output,-1),value_output

def init_weights(layer):
    if isinstance(layer,(nn.Conv2d,nn.Linear)):
        nn.init.orthogonal_(layer.weight)
        nn.init.constant_(layer.bias,0.0)
     
model = network().to(configs.device)
model.forward(
    torch.rand((configs.num_env,configs.num_stack,*(configs.obs_shape)),dtype=torch.float32,device=configs.device)
)
model.apply(init_weights)

with torch.no_grad():
    model.value_head.weight.data *= 100
    model.value_head.bias.data.fill_(20.)
    
model = nn.DataParallel(model)

"""
# Resume training
chk = torch.load("/kaggle/input/1_m190/pytorch/default/1/mario190",map_location=configs.device)
model.load_state_dict(chk["model_state"],strict=True)
model.module.optim.load_state_dict(chk["optim_state"])
"""

'\n# Resume training\nchk = torch.load("/kaggle/input/1_m190/pytorch/default/1/mario190",map_location=configs.device)\nmodel.load_state_dict(chk["model_state"],strict=True)\nmodel.module.optim.load_state_dict(chk["optim_state"])\n'

In [5]:
model.module.value_head.weight

Parameter containing:
tensor([[  3.7680,   9.1783,   4.4187,  22.9356,   5.1605,  -6.7229,   9.5521,
          10.7275,  -6.9100,  -2.8339, -13.7920,  -2.5826,   2.8348,   8.8232,
          17.6081,  26.2936, -26.7314,  -4.0806,  -8.7103,  10.7763,   1.9723,
          13.0598,   4.1047,  -3.6889, -15.5146,  14.4631, -21.3918, -17.3632,
          -1.4463,  -5.0509, -18.0579, -21.6764,  -7.1621,  -4.0634,   7.3665,
           5.7064, -17.4375,  -1.9364,   4.6773,  -7.1996,   9.9093,   7.9119,
         -13.0704, -14.1722,   8.0836,   3.3512,   1.8710, -10.3647,  -9.3560,
           0.2052,  -4.8783, -19.4190,  15.1575,  -6.1900,  -9.8598,   5.6194,
           5.9801,  -4.8462,   2.7027, -12.1573, -17.6636,  12.9699,   3.7151,
          -7.6713,   6.9944,  10.9111,   0.7166,   5.5400,   0.0663,  15.1733,
          -0.7215,  -2.2825,  -0.1588,  10.3585,  26.1236,   4.8838, -18.1959,
           5.8952,  -2.5848,  -6.8745]], requires_grad=True)

In [ ]:
import gc,sys
from torch.distributions import Categorical
from collections import deque

class Memory:
    def __init__(self,env : AsyncVectorEnv):
        self.env = env
        self.gamma = configs.gamma
        self._lambda_ = configs.lambda_
        self.data = []
        self.pointer = 0
        self.episode_reward = np.zeros(self.env.num_envs, dtype=np.float32)
        self.finished_reward = deque(maxlen=5)
        self.log_total_steps = deque(maxlen=5)
        self.total_steps = torch.zeros(configs.num_env)
    
    def rollout(self,batchsize,network : network):
        self.clear()
        self._observation,_ = self.env.reset()
        for n in range(batchsize):
            self._observation = torch.from_numpy(np.array(self._observation)).squeeze(-1).to(configs.device,torch.float32) / 255.
            with torch.no_grad():
                policy_output, value = network.forward(self._observation)
                distribution = Categorical(policy_output)
                action = distribution.sample()
                prob = distribution.log_prob(action)

            state,reward,done,_,_ = self.env.step(action.tolist())

            for i in range(self.env.num_envs): # reward per episode in a deque
                self.episode_reward[i] += reward[i] 
                self.total_steps[i] += 1
                if done[i]:
                    self.finished_reward.append(self.episode_reward[i])
                    self.log_total_steps.append(self.total_steps[i])
                    self.episode_reward[i] = 0
                    self.total_steps[i] = 0
                    
            self.data.append(
                [
                    self._observation,
                    torch.tensor(reward,dtype=torch.float32),
                    value,
                    prob,
                    action,
                    done,
                    distribution
                ]
            )
            self._observation = state
    
        _,rewards,values,_,_,_,_ = zip(*self.data) # advantages 
        _rewards = torch.stack(rewards).to(configs.device) 
        values = torch.stack(values).squeeze(-1) 
        
        next_state = torch.from_numpy(np.array(self._observation)).squeeze(-1).to(configs.device,torch.float32) / 255.
        with torch.no_grad():
            _,next_value = network.forward(next_state)
        _values = torch.cat([values,next_value.permute(-1,0)])
        assert _rewards.dtype == _values.dtype == torch.float32 
        advantages = torch.zeros_like(_rewards,device=configs.device,dtype=torch.float32)
        gae = 0.0
        for n in reversed(range(len(_rewards))):
            td = _rewards[n] + self.gamma * _values[n+1] - _values[n]
            gae = td + (self._lambda_ * self.gamma * gae) 
            advantages[n] = gae
        for data,item in zip(self.data,advantages): # append advantages to data
            data.append(item)
            
        random.shuffle(self.data) 

    def sample(self,number):
        output = self.data[self.pointer:self.pointer+number]
        self.pointer+=number
        states,rewards,values,logProb,actions,done,old_logits,advantages = zip(*output)
        return states,actions,rewards,values,logProb,advantages,old_logits,done

    def clear(self):
        self.data = []
        self.pointer = 0
        gc.collect()
        torch.cuda.empty_cache()
    
    def traj_reward(self):
        return self.finished_reward,self.log_total_steps

In [7]:
from torch.utils.tensorboard import SummaryWriter
from torchviz import make_dot
from torch.distributions.kl import kl_divergence

class agent:
    def __init__(self,):
        self.env = make_env()
        self.network = model
        self.memory = Memory(self.env)
        self.writter = SummaryWriter("./")

    def save(self,k):
        checkpoint = {
            "model_state" : self.network.state_dict(),
            "optim_state" : self.network.module.optim.state_dict(),
        }
        torch.save(checkpoint,f"./mario{k}")

    def train(self,num_game,batchsize,minibatch,optim_steps,graph_render = False):
        for traj in range(num_game):
            self.memory.rollout(batchsize,self.network)

            for _ in range(batchsize//minibatch):
                states,actions,rewards,values,old_log_prob,advantages,old_logits,done= self.memory.sample(minibatch) 
                _advantages = torch.stack(advantages).view(-1)
                _values = torch.stack(values).squeeze().view(-1)
                vtarget = (_advantages + _values)
                assert vtarget.requires_grad == _values.requires_grad == False
                explained_variance = 1.0 - (torch.var(vtarget - _values) / torch.var(vtarget + 1e-10))
                old_logits = [dis.probs for dis in old_logits]
                old_logits = torch.stack(old_logits)
                cat_old_logits = Categorical(probs=old_logits)

                stacked_states = torch.stack(states) # - > [minibatch, num_env, channel, image_shape]
                flat_states = torch.flatten(stacked_states,0,1).to(configs.device) # - > [(minibatch * num_env), channel, image_shape]
                _old_log_prob = torch.stack(old_log_prob).detach()
 
                for _ in range(optim_steps): 
                    p_output,new_values = self.network.forward(flat_states)
                    policy_output = p_output.view(minibatch,configs.num_env,7) # 7 = action space for a single env
                    dist = Categorical(probs=policy_output)  
                    kl = kl_divergence(cat_old_logits,dist).mean() 
                    stacked_actions = torch.stack(actions)  
                    new_log_prob = dist.log_prob(stacked_actions)
                    ratio = torch.exp(new_log_prob - _old_log_prob).view(-1)
                    #norm_advantage  = (_advantages - _advantages.mean()) / (_advantages.std() + 1e-10) 
                    prox1 = ratio * _advantages
                    prox2 = torch.clamp(ratio ,1-configs.epsilon,1+configs.epsilon) * _advantages

                    loss_policy = -torch.mean(torch.min(prox1,prox2))  
                    loss_critic = F.smooth_l1_loss(new_values.squeeze(),vtarget) * configs.critic_coeff
                    _entropy = -(policy_output * torch.log(policy_output + 1e-10)).sum(dim=2)
                    entropy = _entropy.sum(dim=1).mean() * configs.entropy_coeff

                    total_loss = loss_policy + loss_critic - entropy 

                    if graph_render: # computation graph for debugging purposes
                        make_dot(total_loss,dict(self.network.named_parameters())).render("Graph",format="png")
                        graph_render = False
                    
                    self.network.module.optim.zero_grad()
                    total_loss.backward()
                    nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
                    self.network.module.optim.step()
                    
            self.writter.add_scalar("Policy/entropy",entropy,traj)
            self.writter.add_scalar("Policy/loss policy",loss_policy,traj)
            self.writter.add_scalar("Policy/KL",kl,traj)
            self.writter.add_scalar("Value/values",_values.mean(),traj)
            self.writter.add_scalar("Value/vtarget",vtarget.mean(),traj)
            self.writter.add_scalar("Value/value loss",loss_critic,traj) 
            self.writter.add_scalar("Value/Explained variance",explained_variance,traj)
            self.writter.add_scalar("main/total loss",total_loss,traj)
            self.writter.add_scalar("main/epi rewards",torch.tensor([self.memory.traj_reward()[0]]).mean(),traj) 
            self.writter.add_scalar("main/total steps",torch.tensor([self.memory.traj_reward()[1]]).mean(),traj)

            epireward = round(torch.tensor([self.memory.traj_reward()[0]]).mean().tolist(),2)
            print(
                f"{traj}/1k | REWA {epireward} | ENTR {entropy:.2f} | POLI {loss_policy:.2f} | CRIT {loss_critic:.2f} | TLoss {total_loss:.2f} | Val {_values.mean():.2f} | Vtarg {vtarget.mean():.2f} | KL {kl:.4f}"
                )
            if traj % 10 == 0 : # save every ~150k steps
                self.save(traj)
        
        self.save("_end")
    
    @staticmethod
    def run_training(run=False):
        if run:
            agnt = __class__()
            agnt.train(
                num_game=configs.num_game,
                batchsize=configs.batchsize,
                minibatch=configs.minibatch,
                optim_steps=configs.optim_steps
            ) 
            
agent.run_training(True)

SystemExit: torch.Size([5, 4, 100, 100])